In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
DATA_PATH = r"C:\Users\valen\OneDrive\Dokumente\01_Studium\9\Probabilistic ML\Local\player_stats_clean.csv"

# Mindest-Vollständigkeit pro Spalte z. B. 0.7 = 70%
COMPLETENESS_MIN = 0.70

# Korrelation (absolut) ab der Redundanzen entfernen
CORR_MAX = 0.90

df = pd.read_csv(DATA_PATH)

In [3]:
# Spalten definieren und was niemals als Feature genutzt wird
drop_exact = {
    'Player','Pos','Squad','Team','Nation','Comp','Season','League','Rk',
    'Born','Matches','Match','Date','Venue','Round','Opponent','ID','PlayerID'
}
# Alles "Unnamed" etc. raus, auf Nummer sicher gehen
drop_prefixes = ('Unnamed',)

meta_cols = [c for c in df.columns if c in drop_exact or c.startswith(drop_prefixes)]
df_work = df.drop(columns=meta_cols, errors='ignore')

In [4]:
# Domänbasierte Startmenge
# Fokus: per-90 & Progression/Creation & Defensiv-Kern
preferred_features = [
    # per-90 offensive
    'Gls/90','Ast/90','G+A/90','G-PK/90','xG/90','xAG/90','xG+xAG/90','npxG/90','npxG+xAG/90',
    'Sh/90','SoT/90',
    # creation
    'SCA90','GCA90','KP','PPA','1/3','CrsPA','PrgP',
    # progression & receiving
    'PrgC','PrgR',
    # defensive kern
    'Tkl','TklW','Int','Blocks','Clr','Won%','Aerials Won','Aerials Lost',
    # ballbesitz / carries / take-ons
    'Touches','Att 3rd','Mid 3rd','Att Pen','Carries','Succ%','Mis','Dis',
    # effizienz
    'G/Sh','G/SoT','npxG/Sh',
]

# Manche Datensätze nutzen leicht andere Schreibweisen (z.B. Aerial Duels Won, Aerials Won)
aliases = {
    'Aerials Won': ['Aerial Duels Won','Won (Aerials)','Won'],
    'Aerials Lost': ['Lost (Aerials)','Lost'],
    'Won%': ['Aerials Won%','Won% (Aerials)'],
    'Touches': ['Touches (Total)'],
    'Att 3rd': ['Touches (Att 3rd)'],
    'Mid 3rd': ['Touches (Mid 3rd)'],
    'Att Pen': ['Touches (Att Pen)'],
}

# Hilfsfunktion: Feature oder seine Aliase, falls vorhanden
def resolve_feature(name, df_cols):
    if name in df_cols:
        return name
    for k, alist in aliases.items():
        if name == k:
            for alt in alist:
                if alt in df_cols:
                    return alt
    return None

resolved = []
for f in preferred_features:
    got = resolve_feature(f, df_work.columns)
    if got is not None:
        resolved.append(got)

# Zusätzlich ALLE numerischen /90-Spalten automatisch mitnehmen (z. B. SCA90, GCA90 etc.)
auto_per90 = [c for c in df_work.columns if isinstance(c, str) and (('/90' in c) or c.endswith('90'))]

# Numerische Spalten filtern
num_cols = df_work.select_dtypes(include=[np.number]).columns.tolist()

# Kandidaten = (domänenbasiert gefundene + auto_per90) ∩ numerische Spalten
candidates = [c for c in pd.unique(resolved + auto_per90) if c in num_cols]

# Fallback: Wenn durch die Heuristik zu wenig da ist, nimm einfach ALLE numerischen
if len(candidates) < 8:
    candidates = num_cols.copy()

X0 = df_work[candidates].copy()

C:\Users\valen\AppData\Local\Temp\ipykernel_25828\2637413743.py:54: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  candidates = [c for c in pd.unique(resolved + auto_per90) if c in num_cols]


In [5]:
# Datenqualität: Vollständigkeit & simple Imputation
# Vollständigkeit berechnen
completeness = X0.notna().mean()

# Spalten mit zu vielen NaNs entfernen
keep_cols = completeness[completeness >= COMPLETENESS_MIN].index.tolist()
X1 = X0[keep_cols].copy()

# Für verbleibende NaNs median
X1 = X1.fillna(X1.median(numeric_only=True))

In [6]:
# Varianz
variances = X1.var(numeric_only=True)
keep_cols_var = variances[variances > 0.0].index.tolist()
X2 = X1[keep_cols_var].copy()


In [7]:
# Korrelation
corr = X2.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

# Wenn zwei Features redundant sind, behalten mit:
# 1) höherer Vollständigkeit, sonst 2) höherer Varianz
to_drop = set()
for col in upper.columns:
    high = upper.index[upper[col] > CORR_MAX].tolist()
    for h in high:
        c1, c2 = col, h
        # Priorität: Vollständigkeit
        c1_comp = completeness.get(c1, 0)
        c2_comp = completeness.get(c2, 0)
        if c1_comp != c2_comp:
            drop = c1 if c1_comp < c2_comp else c2
        else:
            # Tie-breaker: Varianz
            c1_var = variances.get(c1, 0)
            c2_var = variances.get(c2, 0)
            drop = c1 if c1_var < c2_var else c2
        to_drop.add(drop)

X3 = X2.drop(columns=list(to_drop), errors='ignore')


In [8]:
# Scaling für Clsutering
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X3), columns=X3.columns, index=X3.index)


In [10]:
# Reporting
print("-- Feature-Auswahl Bericht --")
print(f"1) Start-Kandidaten: {len(candidates)} Spalten")
print(f"2) Nach Vollständigkeit (≥ {int(COMPLETENESS_MIN*100)}%): {X1.shape[1]} Spalten")
print(f"3) Nach Varianzfilter: {X2.shape[1]} Spalten")
print(f"4) Nach Korrelationsfilter (>|{CORR_MAX}|): {X3.shape[1]} Spalten")
print("\nGewählte Features (für Clustering):")
for c in X3.columns:
    print(" -", c)

-- Feature-Auswahl Bericht --
1) Start-Kandidaten: 18 Spalten
2) Nach Vollständigkeit (≥ 70%): 15 Spalten
3) Nach Varianzfilter: 15 Spalten
4) Nach Korrelationsfilter (>|0.9|): 14 Spalten

Gewählte Features (für Clustering):
 - SCA90
 - GCA90
 - KP
 - PPA
 - 1/3
 - CrsPA
 - PrgP
 - Int
 - Clr
 - Touches
 - TeamSuccess+/-90
 - TeamSuccess(xG)xG+/-90
 - StandardSh/90
 - StandardSoT/90
